# <center>Column generation methods</center>
### <center>Alfred Galichon (NYU & Sciences Po)</center>
## <center>'math+econ+code' masterclass series</center>
#### <center>With python code examples</center>
© 2018–2023 by Alfred Galichon. Past and present support from NSF grant DMS-1716489, ERC grant CoG-866274 are acknowledged, as well as inputs from contributors listed [here](http://www.math-econ-code.org/team).

**If you reuse material from this masterclass, please cite as:**<br>
Alfred Galichon, 'math+econ+code' masterclass series. https://www.math-econ-code.org/

# The cutting stock problem


A paper maker produces raw rolls of width $W$. There are $N$ raw rolls. 

Customers are placing a number $q_i$ of orders of type $i \in \{1,...,I\}$. In order $i$, length $w_i$ is demanded.


A configuration $j \in \{1,...,J\}$ is a way to cut a raw roll into final ones. 


That is, a configuration $j$ specifies $a_{ij}$ which is the number of times in which order $i$ appears in configuration $j$.

$c_j$ is the cost associated with configuration $j$. Often, it is just one, so in the sequel we shall assume<br>
$c_j = 1$ for all $j$,<br>
i.e. we want to minimize the number of raw rolls.

Let $x_j$ be the number of configurations $j$ used.

The program is<br>
$\min_{x_j \geq 0} \sum_j x_j c_j$<br>
s.t. $\sum_j a_{ij} x_j = q_i$<br>
$x_j$ integer.

This problem is a difficult problem, but we shall study its linear programming relaxation.

## Linear programming relaxation


We will consider the linear programming relaxation, i.e. drop the integrality constraint on $x_j$. The relaxed program is then:\
$\min_{x_j \geq 0} \sum_j x_j c_j$<br>
s.t. $\sum_j a_{ij} x_j = q_i$.

Even though the problem is a linear programming one, it is still difficult because the matrix $A=(a_{ij})$ has many rows, and therefore $J>>I$. 

Let's take an example (taken from Wikipedia):

In [1]:
import numpy as np
w_i = np.array([1380, 1520, 1560, 1710, 1820, 1880, 1930, 2000, 2050, 2100, 2140, 2150, 2200])
q_i = np.array([22,25,12,14,18,18,20,10,12,14,16,18,20])
W = 5600
nbi = len(w_i)

Let us give a first example of a configuration $(a^{ex1}_i)$ with one order $i=0$, one $i=1$, and one $i=2$, and let's test its feasibility:

In [2]:
aex1_i = np.array([1,1,1]+[0]*(nbi-3))
print('aex1_i',aex1_i)
print("Feasible" if aex1_i @ w_i <= W else "Not Feasible")

aex1_i [1 1 1 0 0 0 0 0 0 0 0 0 0]
Feasible


We are going to keep track of a base $j\in B$ of configurations $A_{ij}$, where $A_{ij}$ is the quantity of order $i$ in configuration $j$.

Our intial feasible basis will consist in taking patterns that produce only one type of final rolls, that is for $j\in\{1,...,J\}$, $a_{jj} = \lfloor W / w_j \rfloor$ and $a_{jk}=0$ for $k\neq j$.<br>
The basic solution is then $x^{sol} = A^{-1} w$, that is<br>
$x^{sol}_j=q_j / \lfloor W / w_j \rfloor$ for $j\in\{1,...,J\}$.

In [3]:
A_i_j = np.diag( W // w_i)
xsol_j = np.linalg.solve(A_i_j,q_i)
xsol_j

array([ 5.5       ,  8.33333333,  4.        ,  4.66666667,  6.        ,
        9.        , 10.        ,  5.        ,  6.        ,  7.        ,
        8.        ,  9.        , 10.        ])

The associated objective value is $\sum_j x^{sol}_j$, that is:

In [4]:
xsol_j.sum()

92.5

Now let's go back to our example configuration `aex_i` and let's check if we are making a saving by introducing it in order to replace vectors of the base.

If we introduce one unit of the example configuration, we need to remove $z_j$ of each of the basic configurations $j\in B$, where<br>
$a^{ex1}_i = \sum_{j\in B} A_{ij} z_j$.

This can be expressed as<br>
$a^{ex1} = A z,$<br>
so we have<br>
$z = A^{-1} a^{ex1}.$

The impact on the cost is $1-\sum_j z_j=1 - 1_I^\top A^{-1} a^{ex1} = 1 - k^\top a^{ex1}$<br>
where<br>
$k= (A^{-1})^\top 1_I$.

We let:

In [5]:
k_i = np.linalg.solve(A_i_j.T,np.ones(nbi))

In the case of our example, we have:

In [6]:
1 - k_i.dot(aex1_i)

0.08333333333333348

This means that introducing this configuration would add to the cost. But are there configurations that decrease the cost?

In order to look for a configuration that decreases the cost, we need to compute:<br>
$\max_a \sum_{i \in I} k_i a_i$<br>
s.t. $\sum_{i\in I} a_i w_i \leq W$<br>
$a_i$ integer.<br>
This is called the *knapsack problem*.


## The knapsack problem

Let $F(i,v)$ be the value of the above problem where $W$ is replaced by $v$ and $I$ by $\{1,...,i\}$.  We have the following dynamic programming principle:<br>
$F(i+1,v) = max_a\{k_{i+1}a+F(i,v-w_{i+1}a) : a \in \{ 0,..., \lfloor v / w_{i+1} \rfloor \} \}$<br>
which is initialized by<br>
$F(1,v) =  \lfloor v / w_{1} \rfloor max(k_1,0)$.

In [7]:
def knapsack(k_i,W):
    nbi=len(k_i)
    F = np.zeros( (nbi,W+1))
    A = np.zeros( (nbi,W+1),dtype = int)
    A[0,:]=np.array([(v // w_i[0])  for v in range(W+1)])
    F[0,:]=np.array([(v // w_i[0])* max(k_i[0],0)  for v in range(W+1)])
    for i in range(nbi-1):
        for v in range(W+1):
            thedic = {a: k_i[i+1]*a +F[i,v-a*w_i[i+1]] for a in range( (v// w_i[i+1])+1 )}
            A[i+1,v] = max(thedic, key = thedic.get)
            F[i+1,v] = thedic[A[i+1,v]]
    v = W
    a_i = np.zeros(nbi)
    for i in reversed(range(nbi)):
        a_i[i] = A[i,v]
        v = v-A[i,v]*w_i[i]
    return(a_i)
    

We compute a candidate entering column $a^{ent}_i$. We check that the candidate entering configuration is indeed feasible, and we check that the candidate entering configuration $a^{ent}$ indeed improves on the cost.

In [8]:
aent_i=knapsack(k_i,W)
print('aent_i=',aent_i)
print("Feasible" if aent_i @ w_i <= W else "Not Feasible")
print("Improving" if 1 < k_i.dot(aent_i) else "Not improving")

aent_i= [0. 1. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0.]
Feasible
Improving



If we introduce one unit of the new  configuration, we need to remove $z_j$ of each of the basic configurations $j\in B$, where<br>
$a^{ent}_i = \sum_{j\in B} A_{ij} z^{ent}_j$<br>
so we have<br>
$z^{ent} = A^{-1} a^{ent}.$

We look for $\epsilon$ such that<br>
$\sum_{j \in B} A_{ij}(x_j - \epsilon z^{ent}_j)+\epsilon a^{ent}_i$<br>
has one zero term (which defines the $j$ leaving the basis).<br>
This leads us to determine<br>
$\epsilon = min\{ x_j / z_j : z_j >0 \}$.

In [9]:
zent_j = np.linalg.solve(A_i_j,aent_i)
thedic = {j: xsol_j[j] / zent_j[j] for j in range(nbi) if zent_j[j]>0}
jexit = min(thedic, key = thedic.get)
epsilon = thedic[jexit]
epsilon

9.0

We update the feasible solution and the basis accordingly:

In [10]:
xsol_j = xsol_j - epsilon * zent_j
xsol_j[jexit] = epsilon
print(xsol_j)
A_i_j[:,jexit] = aent_i

[ 5.5         5.33333333  4.          4.66666667  6.          9.
 10.          5.          6.          7.          8.          9.
 10.        ]


We check that the new solution is indeed feasible:

In [11]:
print('xsol_j=\n',xsol_j)
print('\nslackness=',A_i_j @ xsol_j - q_i)

xsol_j=
 [ 5.5         5.33333333  4.          4.66666667  6.          9.
 10.          5.          6.          7.          8.          9.
 10.        ]

slackness= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


The revised objective value is now:

In [12]:
xsol_j.sum()

89.5

## The column generation algorithm

We iterate the previous process and we get:

In [13]:
cont = True
A_i_j = np.diag( W // w_i)
xsol_j = np.linalg.solve(A_i_j,q_i)
iter = 0
while cont:
    iter += 1
    print('Obj=',xsol_j.sum())
    k_i = np.linalg.solve(A_i_j.T,np.ones(nbi))
    aent_i=knapsack(k_i,W)
    if 1 >= k_i.dot(aent_i):
        cont=False
    else:
        zent_j = np.linalg.solve(A_i_j,aent_i)
        thedic = {j: xsol_j[j] / zent_j[j] for j in range(nbi) if zent_j[j]>0}
        jexit = min(thedic, key = thedic.get)
        epsilon = thedic[jexit]
        xsol_j = xsol_j - epsilon * zent_j
        xsol_j[jexit] = epsilon
        A_i_j[:,jexit] = aent_i
        
print( "Converged in "+str(iter)+" steps.\nObjective="+str(xsol_j.sum()))
    

Obj= 92.5
Obj= 89.5
Obj= 86.16666666666667
Obj= 84.5
Obj= 83.0
Obj= 81.25
Obj= 80.91666666666667
Obj= 79.58333333333334
Obj= 78.58333333333334
Obj= 78.25
Obj= 77.91666666666667
Obj= 77.25
Obj= 76.75
Obj= 76.375
Obj= 76.05
Obj= 75.6309523809524
Obj= 75.33333333333333
Obj= 74.5
Obj= 74.16666666666667
Obj= 73.3888888888889
Obj= 73.22222222222221
Obj= 73.0
Obj= 72.98611111111111
Obj= 72.97619047619048
Obj= 72.9423076923077
Obj= 72.92391304347825
Obj= 72.91666666666666
Obj= 72.91666666666667
Converged in 28 steps.
Objective=72.91666666666667
